In [ ]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.integrate

from common.data import load_data

sns.set_theme()

In [ ]:
paths = sorted(Path("../data/5. Battery Data Set/1. BatteryAgingARC-FY08Q4").glob("*.mat"))

for p in paths:
    data = load_data(p, "discharge")

    sns.scatterplot(data=data, x="operation_id", y="Capacity")
    plt.title(Path(p).stem)
    plt.show()

In [ ]:
for p in paths:
    data = load_data(p, "discharge")
    for feature in data.columns[:5]:
        sns.scatterplot(data=data, x="Time", y=feature, hue="Capacity", alpha=0.2)
        plt.title(f"{Path(p).stem}: {feature}")
        plt.show()



In [ ]:
data = pd.concat({Path(p).stem: load_data(p, "discharge") for p in paths})
data.index.names = ["file", "index"]

In [ ]:
grouped_data = data.groupby(["file", "operation_id"]).agg({
    "Time": "max",
    "Capacity": "first",
    "Temperature_measured": "mean",
})

In [ ]:
sns.scatterplot(data=grouped_data, x="Time", y="Capacity", hue="file")
plt.legend([])
plt.show()

In [ ]:
sns.scatterplot(data=grouped_data, x="operation_id", y="Capacity", hue="file")
plt.legend([])
plt.show()

In [ ]:
sns.scatterplot(data=grouped_data, x="Temperature_measured", y="Capacity", hue="file")
plt.legend([])
plt.show()

In [ ]:
cycle_statistics = []
for group, group_data in data.groupby(["file", "operation_id"]):
    power = group_data["Voltage_measured"] * group_data["Current_measured"]
    total_power = scipy.integrate.trapezoid(power, group_data["Time"])
    total_current = scipy.integrate.trapezoid(group_data["Current_measured"], group_data["Time"])
    cycle_statistics.append({
        "file": group[0],
        "operation_id": group[1],
        "total_power": total_power,
        "total_current": total_current,
        "capacity": group_data["Capacity"].iloc[0],
    })
cycle_statistics = pd.DataFrame(cycle_statistics)

In [ ]:
import scipy.stats

In [ ]:
sns.scatterplot(data=cycle_statistics, x="total_power", y="capacity", hue="file")
plt.legend([])
plt.show()
scipy.stats.linregress(cycle_statistics["total_power"], cycle_statistics["capacity"])

In [ ]:
sns.scatterplot(data=cycle_statistics, x="total_current", y="capacity", hue="file")
plt.legend([])
plt.show()
scipy.stats.linregress(cycle_statistics["total_current"], cycle_statistics["capacity"])